In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import os,sys,inspect
# currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# parentdir = os.path.dirname(currentdir)
# parentdir = os.path.dirname(parentdir)
parentdir = "C:\\Users\\cosim\\PycharmProjects\\DDFM"
#parentdir = "/Users/paoloandreini/Desktop/github_repo/DDFM_correction_paper/DDFM"
sys.path.insert(0,parentdir)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load results
# rmsfe_ar = pd.read_csv(f"{parentdir}/examples/empirical/results/rmsfe_ar1_all.csv", index_col=[0,1,2])
# rmsfe_ar.index.names=["variable", "step-ahead", "model"]
# rmsfe_ar.columns =["RMSFE"]
# rmsfe_ar = rmsfe_ar.droplevel("model")
rmsfe_models = pd.read_csv(f"{parentdir}/examples/empirical/results/rmsfe_publ_lag_0_all_final.csv", index_col=[0,1,2])
rmsfe_models.index.names=["variable", "step-ahead", "model"]
rmsfe_models.columns =["RMSFE"]

In [ ]:
# load reuslts ddfm
rmsfe_ddfm = pd.read_csv(f"{parentdir}/examples/empirical/results/ddfms_rmsfe_final.csv")#, index_col=[0,1])
rmsfe_ddfm["Unnamed: 0"] =rmsfe_ddfm["Unnamed: 0"].str.replace("Forecasting Horizon ", "t")
rmsfe_ddfm.columns = ["step-ahead", "variable", "Symmetric", "Asymmetric"]
rmsfe_ddfm = rmsfe_ddfm.set_index(["variable", "step-ahead"])

In [ ]:
rmsfe_all = pd.concat([rmsfe_models.squeeze().unstack("model"), rmsfe_ddfm], axis=1)
rmsfe_all = rmsfe_all.stack()
rmsfe_all.index.names = ["variable", "step-ahead", "model"]

In [ ]:
rmsfe_ratio = (rmsfe_all.unstack("model").T/rmsfe_all.unstack("model")["AR1"]).T
rmsfe_ratio = rmsfe_ratio.drop("AR1", axis=1)

In [ ]:
#rmsfe_ratio= rmsfe_ratio.filter(like="ymm")

In [ ]:
# rmsfe_ratio = (rmsfe_all/rmsfe_ar.squeeze())
# rmsfe_ratio.unstack("model") 

In [ ]:
rmsfe_ratio_per_models = rmsfe_ratio
(rmsfe_ratio_per_models < 1).sum()/len(rmsfe_ratio_per_models)

In [ ]:
rmsfe_ratio_per_models = rmsfe_ratio
(rmsfe_ratio_per_models < 1).sum()/len(rmsfe_ratio_per_models)

In [ ]:
((rmsfe_ratio_per_models < 1).sum()/len(rmsfe_ratio_per_models)).sort_index().sort_values().plot(kind="bar", figsize=(15,5))
plt.axhline(0.5, linestyle="--", color='red', linewidth=0.7)
plt.show()

In [ ]:
rmsfe_ratio_per_models_step = rmsfe_ratio.unstack("step-ahead")
((rmsfe_ratio_per_models_step < 1).sum()/len(rmsfe_ratio_per_models_step)).sort_index()

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(25,6))
ii = 0
for i in ['t1', 't6', 't12', 't24']:#rmsfe_ratio_per_models_step.columns.unique("step-ahead"):
    rmsfe_i = rmsfe_ratio_per_models_step.xs(i, axis=1, level=1)
    #pd.DataFrame(((rmsfe_i <= 1).sum()/len(rmsfe_i)).sort_values()).plot(kind="barh", ax=ax[ii])
    ax[ii].axvline(0.5, linestyle="--", color='red', linewidth=0.7)
    ((rmsfe_i <= 1).sum()/len(rmsfe_i)).sort_values().plot(kind="barh", title=i, ax=ax[ii])
    ii+=1
plt.show()

1. "RPI"= real personal income
2. "INDPRO", =industrial production
3. "IPMANSICS" = industrial production manufacturing
4. "CUMFNS"= capacity utilization
5. "PAYEMS"=Total nonfarm payroll
6. "MANEMP"=Total nonfarm payroll manufactuing
7. "CLAIMSx"=initial claims
8. "UNRATE"=unempl rate
9. "CES2000000008"= Avg Earning Manufacturing
10. "CES3000000008", Avg Earning Construction
11. "DPCERA3M086SBEA"= Real Personal Consumption exepnditure
12. "UMCSENTx"=Consumer Sentiment index
13. "CPIAUCSL"= CPI All: headline
14. "CPIULFSL= CPI Core: Less Food (energy included)
15. retail sales to add



1. change target using h-step ahead
2. abrchart variable each forecast horizon
3. table as Bai/Ng with rmsfe ratio per each step-head.


In [ ]:
important_var = ["RPI", "INDPRO", "IPMANSICS", "CUMFNS", "PAYEMS", "MANEMP", "CLAIMSx", "UNRATE", "CES2000000008", "CES3000000008", "DPCERA3M086SBEA", 
                "UMCSENTx", "CPIAUCSL", "CPIULFSL", "CUSR0000SAC"]

spreads = ['AAAFFM', 'BAAFFM', 'T10YFFM', 'T1YFFM', 'T5YFFM', 'TB3SMFFM',
       'TB6SMFFM']

In [ ]:
pd.set_option('display.max_rows', 150)
rmsfe_var_sel = rmsfe_ratio_per_models_step.loc[important_var]

fig, ax = plt.subplots(1,4, figsize=(25,6))
ii = 0
for i in ['t1', 't6', 't12', 't24']:#rmsfe_ratio_per_models_step.columns.unique("step-ahead"):
    rmsfe_i = rmsfe_var_sel.xs(i, axis=1, level=1)
    ((rmsfe_i <= 1).sum()/len(rmsfe_i)).sort_index().sort_values().plot(title = i, kind="barh", ax=ax[ii])
    ax[ii].axvline(0.5, linestyle="--", color='red', linewidth=0.7)
    ii+=1
plt.show()

## Var Selected

In [ ]:
rmsfe_mdl_min = rmsfe_var_sel.stack("step-ahead")
best_mdl_per_var = []
for i in rmsfe_mdl_min.index:
    min_i = rmsfe_mdl_min.xs(i).min()
    if min_i <=1:
        best_mdl_per_var.append(list(i) + [rmsfe_mdl_min.xs(i)[rmsfe_mdl_min.xs(i) == min_i].index[0]])
    else:
        best_mdl_per_var.append(list(i) + ["AR(1)"])
best_mdl_per_var = pd.DataFrame(best_mdl_per_var, columns=["variable", "step-ahead", "model"])

In [ ]:
best_mdl_per_var.set_index(["variable", "step-ahead"]).unstack("step-ahead")

In [ ]:
ratio_best_mdl = best_mdl_per_var.groupby(["model", "step-ahead"]).count()/len(best_mdl_per_var["variable"].unique())
fig, ax = plt.subplots(1,4,figsize=(25,6))
ii = 0
for step in ['t1', 't6', 't12', 't24']:
    ratio_best_mdl.xs(step, level="step-ahead").sort_values(by="variable").plot(title=step, kind="barh", ax=ax[ii])
    ii+=1
plt.show()

## Improvement

In [ ]:
rmsfe_var_sel.filter(like="t1")

In [ ]:
aa = rmsfe_var_sel # rmsfe_ratio_per_models_step, rmsfe_var_sel
improv_perc = []
for i in aa.columns.unique("model"):
    # print(i, aa[i][aa[i] < 1].mean())
    improv_perc.append((aa[i].mean() -1).values)
    

In [ ]:
pd.DataFrame(improv_perc, index=aa.columns.unique("model"), columns=["t1", "t6", "t12", "t24"])

In [ ]:
aa[i][aa[i] < 1].mean()

## All Variables

In [ ]:
rmsfe_mdl_min = rmsfe_ratio_per_models_step.stack("step-ahead")
best_mdl_per_var = []
for i in rmsfe_mdl_min.index:
    min_i = rmsfe_mdl_min.xs(i).min()
    if min_i <=1:
        best_mdl_per_var.append(list(i) + [rmsfe_mdl_min.xs(i)[rmsfe_mdl_min.xs(i) == min_i].index[0]])
    else:
        best_mdl_per_var.append(list(i) + ["AR(1)"])
best_mdl_per_var = pd.DataFrame(best_mdl_per_var, columns=["variable", "step-ahead", "model"])

In [ ]:
ratio_best_mdl = best_mdl_per_var.groupby(["model", "step-ahead"]).count()/len(best_mdl_per_var["variable"].unique())
fig, ax = plt.subplots(1,4,figsize=(25,6))
ii = 0
for step in ['t1', 't6', 't12', 't24']:
    ratio_best_mdl.xs(step, level="step-ahead").sort_values(by="variable").plot(title=step, kind="barh", ax=ax[ii])
    ii+=1
plt.show()

In [ ]:
rmsfe_ratio_min = rmsfe_ratio.groupby(level=[0,1]).idxmin()["RMSFE"].values.tolist()
rmsfe_ratio_min = pd.DataFrame(rmsfe_ratio_min, columns =['variable', 'step-ahead', 'model'])

In [ ]:
rmsfe_ratio_min.groupby(["model"]).count()["variable"].sort_values().plot(kind='bar', figsize=(10,4))

In [ ]:
# Load results
# rmsfe_ar = pd.read_csv(f"{parentdir}/examples/empirical/results/rmsfe_ar1_all.csv", index_col=[0,1,2])
# rmsfe_ar.index.names=["variable", "step-ahead", "model"]
# rmsfe_ar.columns =["RMSFE"]
# rmsfe_ar = rmsfe_ar.droplevel("model")
mdls_pred = pd.read_csv(f"{parentdir}/examples/empirical/results/bai_ng_results_final_publ_lag0_final.csv", index_col=[0,1,2])
mdls_pred

In [ ]:
mdls_pred.xs("CPIAUCSL").xs("t1").rolling(12, min_periods=1).sum().plot(figsize=(15,10))